In [5]:
import json
import typing
import requests
import pandas as pd

df = pd.read_csv("../data/interim/data.csv")
if 'references_opencitations' in df.columns:
    df.drop(['references_opencitations'], axis=1, inplace=True)
    
api_url = "https://opencitations.net/index/coci/api/v1/references/"

new_column = []

for index, row in df.loc[:, ["doi"]].iterrows():
    
    references = []

    url = f"{api_url}{row.doi}"

    # headers = {"authorization": "YOUR-OPENCITATIONS-ACCESS-TOKEN"}
    headers: typing.Dict[str, str] = {}
    ret = requests.get(url, headers=headers, timeout=300)
    try:
        items = json.loads(ret.text)
        for item in items:
            references.append(item["cited"])
    except json.decoder.JSONDecodeError:
        print(f"Error retrieving citations from OpenCitations for DOI: {row.doi}")
    if len(references) == 0:
        new_column.append(None)
    else:
        new_column.append(references)

df.insert(loc=len(df.columns), column="references_opencitations", value=new_column)
df.to_csv("../data/interim/data.csv", index=False)

    

